In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from catboost import CatBoostRegressor, Pool
from math import erf, sqrt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNetCV
from scipy.stats import norm
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras import regularizers

#from random_forest import RandomForestRegressor

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
#df= pd.read_stata("bl_el_clean.dta")
df_main_old = pd.read_stata("new_cleaned het data.dta")
df_main = pd.read_stata("cleaned het data 2.dta")

In [ ]:
df_main.head(2)

In [ ]:
df_main['_merge'].value_counts()

In [ ]:
df_main.shape

---

# Calculating p(Z) for each household/row

#### First identifying the columns that we will use (columns in the new DF but not in the old one)

In [ ]:
df_main_old_col_names = set(df_main_old.columns)

df_main_col_names = set(df_main.columns)


exlusive_col_names = df_main_col_names- df_main_old_col_names


df_main_exclusive = df_main.loc[:,list(exlusive_col_names)]
df_main_exclusive.head()

##### Combining resticted and full

In [ ]:
df_main['full'] = df_main['full'] + df_main['restricted']
df_main.drop(columns=['restricted'], inplace=True)

In [ ]:
df_main_old_col_names = set(df_main_old.columns)

df_main_col_names = set(df_main.columns)


exlusive_col_names = df_main_col_names- df_main_old_col_names


df_main_exclusive = df_main.loc[:,list(exlusive_col_names)]
df_main_exclusive.head()

In [ ]:
print(df_main[df_main['full'] ==0].shape[0])
print(df_main[df_main['half'] ==1].shape[0])
print(df_main[df_main['control'] ==0].shape[0])

### Combining the full cost and full cost restricted into one full cost variable

In [ ]:
df_main['treatment'].value_counts()

In [ ]:
df_main['treatment'] = np.where(df_main['treatment'].isin(['Full cost','Full cost, restricted']), 'Full cost',df_main['treatment'] )

In [ ]:
df_main['treatment'].value_counts()

In [ ]:
935 + 707

### Calculating p(Z) based on whether the value is in the Full cost, Half cost, or Control group.

In [ ]:
# calculate p(Z) for full vs control
F = df_main['full'].astype(float)     
C = df_main['control'].astype(float)
mask_F_vs_C = df_main['treatment'].isin(['Full cost','Control'])

df_main['p(Z)_full_vs_control'] = np.nan
df_main.loc[(mask_F_vs_C) & (df_main['treatment'].eq('Full cost')), 'p(Z)_full_vs_control'] = (F/(F+C)) # This says for rows in the mask (Full cost, Control), for rows where treatment equals Full cost, set the value of p(Z)_full_vs_control to (F/(F+C))
df_main.loc[(mask_F_vs_C) & (df_main['treatment'].eq('Control')),   'p(Z)_full_vs_control'] = (C/(F+C))

# calculate p(Z) for Hald vs Control
H = df_main['half'].astype(float)     
mask_H_vs_C = df_main['treatment'].isin(['Half cost','Control'])

df_main['p(Z)_half_vs_control'] = np.nan
df_main.loc[(mask_H_vs_C) & (df_main['treatment'].eq('Half cost')), 'p(Z)_half_vs_control'] = (H/(H+C))
df_main.loc[(mask_H_vs_C) & (df_main['treatment'].eq('Control')),   'p(Z)_half_vs_control'] = (C/(H+C))

# calculate p(Z) for Full vs Half
mask_F_vs_H = df_main['treatment'].isin(['Full cost','Half cost'])

df_main['p(Z)_full_vs_half'] = np.nan
df_main.loc[(mask_F_vs_H) & (df_main['treatment'].eq('Full cost')), 'p(Z)_full_vs_half'] = (F/(F+H))
df_main.loc[(mask_F_vs_H) & (df_main['treatment'].eq('Half cost')),   'p(Z)_full_vs_half'] = (H/(F+H))


In [ ]:
print(df_main[df_main['treatment'].isin(['Full cost','Control'])].shape[0])
print(df_main['p(Z)_full_vs_control'].notna().sum())

- Now when we filter by the population inside the function, we will get a p(Z) column for that population. 
- Note that although the p(Z) columns have missing, but after we filter by the population, we will get a p(Z) fir that population with no missing values.

#### Dropping the new not unneeded columns:

In [ ]:
df_main.drop(columns=['half_full','control_trt','full','half','total_n','_merge','control'], inplace=True)

In [ ]:
# List of all p(Z) variables
p_z_list = ['p(Z)_full_vs_control','p(Z)_half_vs_control','p(Z)_full_vs_half']

---

# Transforming the column that will be used for CLAN:

from sklearn.preprocessing import StandardScaler

clan_cols = [
    "treated_HHs","share_all","small_ls_bl","medium_ls_bl","large_ls_bl","w05_ls_value",
    "fattening_ls_count","sold_ls_count","consumed_ls_count","dwelling_rent","water_source_dist",
    "hh_share_latrine","home_improve_amount","ratio_smart_mobil_16","ratio_skiptimes_p_17",
    "ratio_eatoutside__24","nonfood_exp_mon_bl","food_exp_bl","nonagri_sumcapital",
    "agri_own_size_quir","agri_rent_size_quir","agri_rent_amount_bl","agri_rent_out_siz_28",
    "agri_rent_out_amount","bl_fruit_trees_count","totalincome_witho_32","totalincome_1mo",
    "propdisabled_ill","use_time_rev_30da_35","use_time_edu_30da_36","health_cant_work__61",
    "genderrole_mean","hh_prop_prepplus","hh_prop_enrolled","hh_max_attain","hh_size",
    "hh_n_work","dep_ratio","hh_female_prop","own_value_ls","log_totalincome_1mo",
    "elset_sh_eq","inflated_ls_hh","tot_exp_2"
]

scaler = StandardScaler()
df[[f"{c}_std" for c in clan_cols]] = scaler.fit_transform(df[clan_cols]).copy()



---

## Y values

In [ ]:
#Removing the amout that we given to the treatment group from their reported values
df_main['own_value_ls'] = np.where(df_main['treatment'] == 1,df_main['own_value_ls'] - 11379.31, df_main['own_value_ls'] )

##### **inflated_ls_hh is a Y variable and only has 3 unique float values?**

In [ ]:
print(df_main['inflated_ls_hh'].nunique())
df_main['inflated_ls_hh'].value_counts()

#### totalincome_1mo

In [ ]:
print(df_main['totalincome_1mo'].nunique())
print(df_main['totalincome_1mo'].value_counts())
print(f"% of non-0 values: {df_main[df_main['totalincome_1mo'] !=0].shape[0] / df_main.shape[0]}")

In [ ]:
print(df_main.shape[0])
print(df_main['totalincome_1mo'].nunique())
df_main['totalincome_1mo'].value_counts()


In [ ]:
plt.hist(df_main['totalincome_1mo'])

In [ ]:
print(df_main['own_value_ls'].nunique())
print(df_main['own_value_ls'].value_counts())
print(f"% of non-0 values: {df_main[df_main['own_value_ls'] !=0].shape[0] / df_main.shape[0]}")

In [ ]:
plt.hist(df_main['own_value_ls'],bins=30)

In [ ]:
print(df_main['tot_exp_2'].nunique())
print(df_main['tot_exp_2'].value_counts().sort_values())
#print(f"% of non-0 values: {df_main[df_main['tot_exp_2'] !=0].shape[0] / df_main.shape[0]}")

In [ ]:
plt.hist(df_main['tot_exp_2'],bins=30)

In [ ]:
print(df_main['elset_sh_eq'].nunique())
print(df_main['elset_sh_eq'].value_counts().sort_values(ascending=False))
print(f"% of non-0 values: {df_main[df_main['elset_sh_eq'] !=0].shape[0] / df_main.shape[0]}")

In [ ]:
plt.hist(df_main['elset_sh_eq'],bins=100)

In [ ]:
#df_main['elset_sh_eq'] = np.log1p(df_main['elset_sh_eq'])
#df_main['totalincome_1mo'] = np.log1p(df_main['totalincome_1mo'])

In [ ]:
#df_main['own_value_ls'] = np.log1p(df_main['own_value_ls'])

In [ ]:
df_main['own_value_ls'].describe()

In [ ]:
df_main['totalincome_1mo'].describe()

In [ ]:
df_main['own_value_ls'].describe()

In [ ]:
df_main[df_main['elset_sh_eq'] > 30].describe()

In [ ]:
df_main['elset_sh_eq'].describe()

In [ ]:
df_main[df_main['elset_sh_eq'] > 30].shape[0]

In [ ]:
df_main[(df_main['elset_sh_eq'].rank(pct=True) >= 0.90) & (df_main['elset_sh_eq']>=100)].shape[0]


In [ ]:
print(f"{df_main[df_main['elset_sh_eq'].rank(pct=True).between(0.95, 1)]['own_value_ls'].mean()}")
print(f"{df_main[df_main['elset_sh_eq'].rank(pct=True).between(0.95, 1)]['own_value_ls'].median()}")

In [ ]:
df_main['elset_sh_eq'].quantile(0.95) #filter by 95th percentile only for this variable run

In [ ]:
df_main['elset_sh_eq'].quantile(0.95) / df_main['elset_sh_eq'].shape[0]

In [ ]:
plt.hist(df_main['elset_sh_eq'],bins=100)

In [ ]:
all_y_list = ["own_value_ls","totalincome_1mo","elset_sh_eq","tot_exp_2"]

In [ ]:
# Log of the y, Experimental:
#all_y_list = ["own_value_ls","totalincome_1mo","elset_sh_eq","tot_exp_2","inflated_ls_hh"]

#df_main['totalincome_1mo'] = np.log1p(df_main['totalincome_1mo'])

---

### Cleaning ID and other irrelevant columns

In [ ]:
# Chcecking all cols types
unique_dtypes = df_main.dtypes.map(lambda dt: dt.name).unique()
print(unique_dtypes)

In [ ]:
object_columns = df_main.select_dtypes(include=['object']).columns
object_columns 

In [ ]:
df_main.drop(columns=['hhid', 'hhid_id', 'agg_id', 'q_id','nn','inflated_ls_hh'], inplace=True)
object_columns = df_main.select_dtypes(include=['object']).columns
object_columns 

## Categorical Variables Cleaning

In [ ]:
df_main['ladder'].value_counts()

In [ ]:
# 1) Drop the row(s) with ladder == 2.077067 
df_main = df_main[~df_main["ladder"].astype(str).str.strip().eq("2.077067")].copy()

# 2) Convert labels like "1 - extremly poor living" and "10 - very comfortable living" to integers
df_main["ladder"] = df_main["ladder"].astype(str).str.extract(r"^\s*(\d+)")[0].astype(int)

#### Converting the binary columns from string to int

In [ ]:
df_main.head(3)

In [ ]:
yes_no_cols = ['own_ls','ls_stable','latrine','worry_about_food','got_medical_assist',
               'agri_own_yesno', 'agri_rent_yesno', 'agri_rent_out_yesno', 'hhmembers_agri_work', 
               'cart_e', 'bicycle_e', 'vehicle_toktok_e', 'vehicle_tricycle_e', 'vehicle_motorcycle_e', 
               'has_fridge_e','skip_meal_e','has_heater_e']

for col in yes_no_cols:
    df_main[col] = np.where(df_main[col] == 'Yes', 1, 0)

In [ ]:
df_main.head(3)

In [ ]:
print(df_main['haschld'].value_counts())
print()
print(df_main['illithd'].value_counts())
print()
print(df_main['hhdisabl_chld'].value_counts())
print()
print(df_main['hhdisabl_adult'].value_counts())

In [ ]:
df_main['femhd'].value_counts()

In [ ]:
binary_string_cols = ['haschld', 'illithd', 'hhdisabl_chld', 'hhdisabl_adult','oldhd','younghd','singhd']

for col in binary_string_cols:
    df_main[col] = (pd.to_numeric(df_main[col], errors="coerce") != 0).astype("int8")

df_main['femhd'] = np.where(df_main['femhd'] == 'female HH-head', 1, 0)

In [ ]:
print(df_main['haschld'].value_counts())
print()
print(df_main['illithd'].value_counts())
print()
print(df_main['hhdisabl_chld'].value_counts())
print()
print(df_main['hhdisabl_adult'].value_counts())
print()
print(df_main['oldhd'].value_counts())
print()
print(df_main['younghd'].value_counts())
print()
print(df_main['singhd'].value_counts())
print()
print(df_main['femhd'].value_counts())

### Clean some binary columns to int instead of float

#### Ask about this variable:

In [ ]:
df_main.head(3)

In [ ]:
# columns with exactly two unique values
binary_cols = [c for c in df_main.columns if df_main[c].nunique() == 2]

for col in binary_cols:
    df_main[col] = df_main[col].astype("Int64")

### Defining a list of all potential target variables to be used as a parameter in the function

In [ ]:
all_y_list = ["own_value_ls","totalincome_1mo","elset_sh_eq","tot_exp_2"]

---

In [ ]:
df_main['treatment'].value_counts()

In [ ]:
# List of all p(Z) variables
p_z_list = ['p(Z)_full_vs_control','p(Z)_half_vs_control','p(Z)_full_vs_half']

In [ ]:
def estimate_heterogeneity(df_main, y, population_1, population_2, n_splits, ml_model, binary_cols, all_y_list):
    """
    df: pandas.DataFrame
    y: str  (e.g., "own_value_ls")
    population_1, population_2: values in df['treatment'] to compare (pop1=1, pop2=0)
    n_splits: int
    ml_model: str or estimator (e.g., "catboost", "xgboost", "lightgbm", "rf", or a fitted-like object)
        Currently we have "catboost", "xgboost", "nn" options.
    all_y_list: list of strings that contains all the options for target variables, after passing the wanted target variable, the rest will be dropped 
    """

    #p_of_z = 1.0/2.0 #Delete later
    binary_cols = binary_cols

    df_population = df_main.copy() 
    # Defining the DataFrame:
    df_population = df_population[df_population["treatment"].isin([population_1, population_2])].copy()
    df_population["D"] = np.where(df_population["treatment"] == population_1, 1, 0)  # pop1=1, pop2=0

    # Defining p(Z):
    if {population_1, population_2} == {"Full cost", "Control"}:
        df_population.drop(columns=['p(Z)_half_vs_control','p(Z)_full_vs_half'], inplace=True)
        df_population["p(Z)"] = df_population['p(Z)_full_vs_control']

    elif {population_1, population_2} == {"Half cost", "Control"}:
        df_population.drop(columns=['p(Z)_full_vs_control','p(Z)_full_vs_half'], inplace=True)
        df_population["p(Z)"] = df_population['p(Z)_half_vs_control']

    elif {population_1, population_2} == {"Full cost", "Half cost"}:
        df_population.drop(columns=['p(Z)_full_vs_control','p(Z)_half_vs_control'], inplace=True)
        df_population["p(Z)"] = df_population['p(Z)_full_vs_half']


    # Dropping the treatment column:
    df_population.drop(columns=["treatment"], inplace=True)


    # Results container:
    results = {"blp": [], "gates": [], "clan": []}
    models_comparisons = {"small_lambda_blp":[], "small_lambda_gates":[]}  # to store the models comparison results. BLP and GATES small Lambdas

    
    # Dropping the columns that are not features:
    # list of all Y columns
    # Option A: copy + remove
    all_y_list = all_y_list.copy()
    others_y_list = all_y_list.copy()
    others_y_list.remove(y)              # y must be in the list
    df_population = df_population.drop(columns=others_y_list)

    # Creating a dictionary to store ML proxy predictions
    s_records = {idx: [] for idx in df_population.index} #dict with keys as the idx, and for each one, we have an empty list to each loop's prediction per idx/row.

    #-------------------------------------------------------------------------------------------------------------------

    # Big loop:
    base_seed = 7
    for split in range(n_splits):
        df = df_population.copy() # Reformating the original DF for each split so the split specific columns are re-defined

        # D-stratified 50/50 split → persistent 'fold' column
        # first assign all values to main
        df["fold"] = "main" 
        # Then assign half of each group of D (0 & 1) to aux randomly.
        aux_idx = (df.groupby("D", group_keys=False)
               .apply(lambda g: g.sample(frac=0.5, random_state= base_seed + split*10))).index   # group_keys=False
        df.loc[aux_idx, "fold"] = "aux"

    #-------------------------------------------------------------------------------------------------------------------
        # ML Proxy:
        # Defining the data for the two ML models:
        u_0_data = df[(df['fold'] == 'aux') & (df['D'] == 0)].copy()
        u_1_data = df[(df['fold'] == 'aux') & (df['D'] == 1)].copy()

        # 1) Define X, y for each aux subset
        drop_cols = [y, "D", "fold", "p(Z)"]

        X_0 = u_0_data.drop(columns=drop_cols)
        y_0 = u_0_data[y]

        X_1 = u_1_data.drop(columns=drop_cols)
        y_1 = u_1_data[y]

        # Decision the ML model to use
       # if ml_model == 'catboost':
        #    alpha = 0.45
        #    cb_params = dict(
        #        iterations=700,
        #        depth=8,
        #        learning_rate=0.06,
        #        loss_function=f"Quantile:alpha={alpha}",
        #        #loss_function="Huber:delta=2000",
        #        eval_metric=f"SMAPE",           # robust eval
        #        bootstrap_type="Bernoulli",
        #        subsample=0.8,
        #        rsm=0.8,
        #        l2_leaf_reg=6.0,
        #        od_type="Iter", od_wait=60,     # early stopping
        #        random_seed=39,
        #        verbose=False,
        #        allow_writing_files=False,
        #    )


#-----------------------------------------------------------------------

        if ml_model == 'catboost': 
            #alpha = 0.9
            cb_params = dict(
                iterations= 700, 
                depth= 8, 
                learning_rate= 0.05, 
               # loss_function= f"Quantile:alpha={alpha}", 
                #loss_function = 'RMSE',
                loss_function = 'Huber:delta=1000',
                #eval_metric= f"SMAPE", 
                #robust eval bootstrap_type="Bernoulli",
                #subsample=0.8, rsm=0.8, l2_leaf_reg=6.0,
                #od_type="Iter",
                #od_wait=60,
                # early stopping random_seed=39,
                verbose=False,
                # allow_writing_files=False,

        )

            # fit
            model_u1 = CatBoostRegressor(**cb_params).fit(Pool(X_1, y_1, cat_features=binary_cols))
            model_u0 = CatBoostRegressor(**cb_params).fit(Pool(X_0, y_0, cat_features=binary_cols))

        elif ml_model == 'xgboost':
            xgb_params = dict(
                n_estimators=700,
                max_depth=8,
                learning_rate=0.1,
                #subsample=0.8,
                #colsample_bytree=0.8,
                #random_state=39,
                #n_jobs=-1,
                objective="reg:squarederror",
                #enable_categorical=True
            )

            model_u1 = XGBRegressor(**xgb_params).fit(X_1, y_1)  # μ̂1(Z)
            model_u0 = XGBRegressor(**xgb_params).fit(X_0, y_0)  # μ̂0(Z)
            
        # Creating a RF option
        elif ml_model == 'rf':
            rf_params = dict(
                n_estimators=300,
                max_depth=8,
                max_features="sqrt",   # similar to colsample_bytree
                min_samples_leaf=1, # minimum number of samples required to be in a leaf node
                #n_jobs=-1, # ontrols how many CPU cores scikit-learn will use in parallel (n_jobs=-1: use all available cores.)
                #random_state=39
            )

            model_u1 = RandomForestRegressor(**rf_params).fit(X_1, y_1)  # μ̂1(Z)
            model_u0 = RandomForestRegressor(**rf_params).fit(X_0, y_0)  # μ̂0(Z)

        # Neural Network Option:
        elif ml_model == 'nn':
            # to float32
            X1 = np.asarray(X_1, dtype="float32"); y1 = np.asarray(y_1, dtype="float32")
            X0 = np.asarray(X_0, dtype="float32"); y0 = np.asarray(y_0, dtype="float32")

            model_u1 = keras.Sequential([
                layers.Input(shape=(X1.shape[1],)),
                layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l2(1e-4)),
                #layers.BatchNormalization(),
                layers.Dense(64, activation="relu", kernel_regularizer=regularizers.l2(1e-4)),
                #layers.BatchNormalization(),
                layers.Dense(32, activation="relu", kernel_regularizer=regularizers.l2(1e-4)),
                #layers.BatchNormalization(),
                layers.Dense(16, activation="relu", kernel_regularizer=regularizers.l2(1e-4)),
                #layers.BatchNormalization(),
                layers.Dense(1)
                ])
            model_u1.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3, clipnorm=1.0),
                            loss="mse",
                            metrics=[keras.metrics.RootMeanSquaredError()]
                            )

            model_u0 = keras.Sequential([
                layers.Input(shape=(X0.shape[1],)),
                layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l2(1e-4)),
                #layers.BatchNormalization(),
                layers.Dense(64, activation="relu", kernel_regularizer=regularizers.l2(1e-4)),
                #layers.BatchNormalization(),
                layers.Dense(32, activation="relu", kernel_regularizer=regularizers.l2(1e-4)),
                #layers.BatchNormalization(),
                layers.Dense(16, activation="relu", kernel_regularizer=regularizers.l2(1e-4)),
                #layers.BatchNormalization(),
                layers.Dense(1)
            ])
            model_u0.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3, clipnorm=1.0),
                            loss="mse",
                            metrics=[keras.metrics.RootMeanSquaredError()]
                            )

            # fit  (keep variables as the MODEL, not the History)
            model_u1.fit(X1, y1, epochs=50, batch_size=64, verbose=0)
            model_u0.fit(X0, y0, epochs=50, batch_size=64, verbose=0)

            ### Trying an Elastic Net Model option:
            # Elastic Net option:
        elif ml_model == 'elasticnet':
            enet_params = dict(
                l1_ratio=[0.1, 0.3, 0.5, 0.7, 0.9],  # search over mix of L1/L2
                n_alphas=100,
                cv=5,
                random_state=39,
                max_iter=5000,
                n_jobs=-1
            )
            model_u1 = Pipeline([
                ("scaler", StandardScaler()),
                ("enet", ElasticNetCV(**enet_params))
            ]).fit(X_1, y_1)
            model_u0 = Pipeline([
                ("scaler", StandardScaler()),
                ("enet", ElasticNetCV(**enet_params))
            ]).fit(X_0, y_0)


        #elif ml_model == 'xyz': Place holder to add other model options later.

        #------------------------------------------------------------------------------------------------------


        # drop list you used when training
        drop_cols = [y, "D", "fold", "p(Z)"]

        # Applying the predictions to the main fold:
        # main fold
        main = df[df["fold"] == "main"].copy()

        # features for main (must match training: drop non-features)
        X_main = main.drop(columns=drop_cols).copy()


        if ml_model == 'nn':
            X_pred = X_main.reindex(columns=X_1.columns, fill_value=0).to_numpy(dtype="float32")
            model_u1_hat = np.asarray(model_u1.predict(X_pred)).reshape(-1)
            model_u0_hat = np.asarray(model_u0.predict(X_pred)).reshape(-1)
        else:
            # predict μ̂1 and μ̂0 on main, then S = μ̂1 − μ̂0 on any other model
            model_u1_hat = model_u1.predict(X_main)
            model_u0_hat = model_u0.predict(X_main)

        # we filter df by the location of the X_main, and then create a new column
        df.loc[main.index, "S(Z)"] = model_u1_hat - model_u0_hat

        # Also adding the base line preidictions B(Z):
        df.loc[main.index, "B(Z)"] = model_u0_hat


        # Collect S(Z) for rows that are in 'main' this split
        for idx, sval in df.loc[main.index, "S(Z)"].items():
            if pd.notnull(sval):
                s_records[idx].append(float(sval))

        #---------------------------------------------------------------------------------------------
        # BLP:

        # First Creating some BLP equation columns:
        # Creating the D-p column:
        mask = df['fold'].eq('main')
        df['D-p'] = np.nan
        #df.loc[mask, 'D-p'] = df.loc[mask, 'D'].astype(float) - float(p_of_z) 
        df.loc[mask, 'D-p'] = df.loc[mask, 'D'].astype(float) - df.loc[mask, 'p(Z)'].astype(float)


        # Create the shifted (centered) ML proxy on the MAIN sample
        mask = df['fold'].eq('main')
        df['S(Z)_shifted'] = np.nan
        mu_S = df.loc[mask, 'S(Z)'].mean()
        df.loc[mask, 'S(Z)_shifted'] = df.loc[mask, 'S(Z)'] - mu_S

        # Creating a single (D-p)*shifted_S_xgb column
        mask = df['fold'].eq('main')
        df['D-p_S(Z)_shifted'] = np.nan
        df.loc[mask, 'D-p_S(Z)_shifted'] = df.loc[mask, 'D-p'].astype(float) * df.loc[mask, 'S(Z)_shifted'].astype(float)


        # BLP Model:
        
        mask = df['fold'].eq('main')

        y_vec = df.loc[mask, y]  
        X = df.loc[mask, ['D-p', 'D-p_S(Z)_shifted', 'B(Z)', 'p(Z)']]

        X = sm.add_constant(X)  # adds the intercept α

        blp = sm.OLS(y_vec, X).fit(cov_type='HC1')  # robust SEs

        # Saving the BLP results:
        # We will save the R-Squared, R-Squared Adjusted, p-value, and the coefficients for D-p and  D-p_S(Z)_shifted.
        # column keys must match exactly how you named them in X


        beta2 = blp.params['D-p_S(Z)_shifted']      # slope on (D-p) * (S - mean_S)
        S_main = df.loc[mask, 'S(Z)'].to_numpy()    # raw S on the main split (not standardized)
        #var_y_main = float(np.var(df.loc[mask,y], ddof=0)) # population var to divide by

        small_lambda_blp = float(beta2**2) * float(np.var(S_main, ddof=0)) 

        #small_lambda_blp = float(beta2**2) * float(np.var(S_main, ddof=0))  / var_y_main


        # inside the loop, AFTER fitting `blp`
        k1, k2 = "D-p", "D-p_S(Z)_shifted"

        def grab(key):
            lo, hi = blp.conf_int().loc[key]
            return {
                "name": key,
                "coef": blp.params[key],
                "std_err": blp.bse[key],
                "z": blp.tvalues[key],
                "p": blp.pvalues[key],
                "ci_low": lo,
                "ci_high": hi,
            }

        split_blp = [
            {"name": "D_minus_p", **grab(k1)},
            {"name": "D_minus_p_times_S", **grab(k2)},
        ]

        beta2 = blp.params['D-p_S(Z)_shifted']      # slope on (D-p) * (S - mean_S)
        S_main = df.loc[mask, 'S(Z)'].to_numpy()    # raw S on the main split (not standardized)

        #small_lambda_blp = float(beta2**2) * float(np.var(S_main, ddof=0))  # population var
        small_lambda_blp = float(beta2**2) * float(np.var(S_main, ddof=1))


        results["blp"].append(split_blp)   
        models_comparisons["small_lambda_blp"].append(small_lambda_blp)
        
    #return results # Only used for debugging
        

       #-----------------------------------------------------------------------------------------------
       # GATES

       # We will start by creating a new column that assigns quantile groups to each S(Z) value.
       # MAIN-only groups for GATES
        mask_main = df["fold"].eq("main")
        s_main = df.loc[mask_main, "S(Z)"].dropna()

        df["Group"] = np.nan
        df.loc[mask_main, "Group"] = (
            pd.qcut(s_main, q=5, labels=[1,2,3,4,5], duplicates="drop")
            .astype(float)
        )

        # Creating the main column for the GATES model, which is T_K
        # T_k = (D - p) * 1{Group=k}
        for k in range(1, 6):
            df.loc[mask_main, f"T{k}"] = (
                df.loc[mask_main, "D-p"] *
                (df.loc[mask_main, "Group"].astype(int) == k).astype(int)
            )


        # Fit GATES Model:
        dfm = df.loc[mask_main, [y, "T1","T2","T3","T4","T5","B(Z)"]].dropna()
        X = sm.add_constant(dfm[["T1","T2","T3","T4","T5","B(Z)"]], has_constant="add")
        gates = sm.OLS(dfm[y], X).fit(cov_type="HC1")
        params = gates.params


        # Saving the GATES results in our results dictionary:
        ci = gates.conf_int()  # 95% by default
        params = gates.params
        bse = gates.bse

        split_gates = []
        for k in range(1, 6):
            tk = f"T{k}"
            split_gates.append({
                "group": k,
                "gamma": float(params[tk]),
                "std_err": float(bse[tk]),
                "ci_low": float(ci.loc[tk, 0]),
                "ci_high": float(ci.loc[tk, 1]),
            })

        results["gates"].append(split_gates)

        bar_lambda = 0
        for k in range(1, 6):
            tk = f"T{k}"
            bar_lambda += float(gates.params[tk])**2 
        
        #bar_lambda_gates = (bar_lambda / 5.0) / var_y_main
        bar_lambda_gates = (bar_lambda / 5.0) 


        models_comparisons["small_lambda_gates"].append(bar_lambda_gates)

        

       #-----------------------------------------------------------------------------------------------
       # CLAN
        # Steps:
            #1. Get only the numeric columns.
            #2. Remove all the columns created in the previous steps and the target column.
            #3. Remove all the columns that numericly coded but are binary.
            #4. Remove additional columns that are categorical but were not detected as binary (found after in the CLAN table)
            #5. Getting a list of all columns that will be entered to CLAN

        # 1. Keeping only numeric columns
        df_numeric = df.select_dtypes(include=[np.number])

        # 2. Removing the columns created in the previous steps
        clan_exclude = [
            "D",
            "S(Z)",
            "D-p",
            "S(Z)_shifted",
            "D-p_S(Z)_shifted",
            "B(Z)",
            "p(Z)",
            "Group",
            "T1", "T2", "T3", "T4","T5",
            y
        ]
        # 3. Removing binary columns:
        binary_cols_01 = [c for c in df_numeric.columns
                        if set(df_numeric[c].dropna().unique()) == {0, 1}]

        #4.Remove additional columns that are categorical but were not detected as binary (found after in the CLAN table)
        additional_features_drop = ['vehicle_motorcycle_e','vehicle_toktok_e','vehicle_tricycle_e','skip_meal_e','has_heater_e','has_fridge_e','cart_e','bicycle_e']

        #5. Getting a list of all columns that will be entered to CLAN
        # Combing all exclude column lists and filter the numeric DF:
        cols_to_remove = clan_exclude + binary_cols_01 + additional_features_drop

        # filtering the numeric DF to get only a DF with only useful columns
        df_clan_cols = df_numeric.drop(columns=cols_to_remove)

        # This is our final feature list
        clan_cols = df_clan_cols.columns

        # Creating a Clan Table
        # 1. Filter to inlcude only the main fold
        # 2. Include only the CLAN columns + the group column
        # 3. Include only group 1 and 5

        # filtering to include only main fold
        df_main = df[df['fold'] == 'main']

        # Filtering to include only CLAN columns + group
        df_clan = df_main.loc[:, list(clan_cols) + ["Group"]].copy()

        # keep only groups 1 and 5 (fixing the var name)
        df_clan = df_clan[df_clan["Group"].isin([1, 5])]



        #  Calculating the means, the diff, the SE, the z, the p, the CI for each variable
        stats = (
            df_clan.groupby("Group")[clan_cols]
            .agg(["mean", "std", "count"])
            .reindex([1, 5])  # ensure order: G1, G5
        )

        rows = []
        zcrit = 1.96  # 95% CI using normal critical value

        for col in clan_cols:
            m1 = stats.loc[1, (col, "mean")]
            m5 = stats.loc[5, (col, "mean")]
            s1 = stats.loc[1, (col, "std")]
            s5 = stats.loc[5, (col, "std")]
            n1 = stats.loc[1, (col, "count")]
            n5 = stats.loc[5, (col, "count")]

            diff = m5 - m1
            
            # Welch SE for difference in means
            se = np.sqrt((s5**2) / n5 + (s1**2) / n1) 
            z  = diff / se

            # Two-sided p-value from the normal CDF via erf
            # Phi(z) = 0.5 * (1 + erf(z / sqrt(2)))
            
            p_two_sided = 2 * (1 - (0.5 * (1 + erf(abs(z) / sqrt(2)))))
        

            ci_lo = diff - zcrit * se 
            ci_hi = diff + zcrit * se 

            # save the results to our results dictionary
            rows.append({
                "covariate": col,
                "Mean G1 (predicted least affected)": m1,
                "Mean G5 (predicted most affected)": m5,
                "Diff (G5 - G1)": diff,
                "SE (Welch)": se,
                "z": z,
                "p (two-sided)": p_two_sided,
                "CI 95% lower": ci_lo,
                "CI 95% upper": ci_hi,
            })

        # save CLAN for this split (one big list of covariate rows)
        results["clan"].append(rows)

        # End of the loop

        # Calculating the median S(Z) for each row/list
        s_median_series = pd.Series(
            {idx: (np.median(vals) if len(vals) > 0 else np.nan) for idx, vals in s_records.items()}
        ).reindex(df_population.index)

        # If you specifically want a plain list:
        s_medians_per_row = s_median_series.tolist()



    #------------------------------------------------------------------------------------------------
    return results, models_comparisons, s_medians_per_row # the final result dictionaries


In [ ]:
results, models_comparisons, s_medians_per_row = estimate_heterogeneity(df_main, 'own_value_ls', 'Full cost', 'Control', n_splits=100, ml_model='catboost', binary_cols=binary_cols, all_y_list=all_y_list)

all_y_list = ["own_value_ls","totalincome_1mo","elset_sh_eq","tot_exp_2"]

---

### Plot the distribution of S(Z) (Median S(Z) per row)

In [ ]:
plt.hist(s_medians_per_row,bins=100)

In [ ]:
import matplotlib.ticker as ticker


plt.figure(figsize=(12, 6))
plt.hist(s_medians_per_row, bins=100)

# Format x-axis to use 'k' notation
plt.gca().xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, p: f'{x/1000:.0f}k' if x >= 1000 else f'{x:.0f}'))

# Manually set the tick positions for even distribution
x_min, x_max = plt.xlim()
custom_ticks = np.linspace(x_min, x_max, 30)  # 15 ticks across the range
plt.xticks(custom_ticks, rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.kdeplot(s_medians_per_row,shade=True)
# Format x-axis to use 'k' notation
plt.gca().xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, p: f'{x/1000:.0f}k' if x >= 1000 else f'{x:.0f}'))

# Manually set the tick positions for even distribution
x_min, x_max = plt.xlim()
custom_ticks = np.linspace(x_min, x_max, 30)  # 15 ticks across the range
plt.xticks(custom_ticks, rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
plt.hist(df_main['own_value_ls'], bins=100)

In [ ]:
df_main['own_value_ls'].describe()

In [ ]:
s_median_dict = {'s_median': s_medians_per_row}
s_median_df = pd.DataFrame(s_median_dict)
s_median_df.describe()

---

## **First let's Compare the models**

In [ ]:
small_lambda_blp = np.median(models_comparisons['small_lambda_blp'])
small_lambda_gates = np.median(models_comparisons['small_lambda_gates'])
print(F"small_lambda_blp: {small_lambda_blp.round(0)}, small_lambda_gates: {small_lambda_gates.round(0)}")

NN: small_lambda_blp: 51879186.868425295, small_lambda_gates: 188142249.3749832


In [ ]:
small_lambda_blp_normalized = np.median(models_comparisons['small_lambda_blp']) / df_main['own_value_ls'].var()
small_lambda_gates_normalized = np.median(models_comparisons['small_lambda_gates']) / df_main['own_value_ls'].var()
print(F"small_lambda_blp_normalized: {small_lambda_blp_normalized}, small_lambda_gates_normalized: {small_lambda_gates_normalized}")

- elasticnet: lambda_blp_normalized: 0.15897482512187378, lambda_gates_normalized: 0.5596844870804755
- xgboost: lambda_blp_normalized: 0.10458710928863613, lambda_gates_normalized: 0.49125177179907553
- Random Forest: lambda_blp_normalized: 0.243244078015595, lambda_gates_normalized: 0.5830742598858208
- catboost_100: lambda_blp_normalized: 0.21525394117555752, lambda_gates_normalized: 0.4747823367888713
- NN: lambda_blp_normalized: 0.08692043861869532, lambda_gates_normalized: 0.388215865677688

In [ ]:
df_main['own_value_ls'].var()

---

## **Second: For the best model, extract the results**

In [ ]:
results

In [ ]:
rows = []
for run_i, run in enumerate(results['blp'], start=1):
    for d in run:
        if 'name' in d:  # only coefficient rows
            rows.append({
                'run': run_i,
                'name': d['name'],
                'coef': float(d['coef']),
                'std_err': float(d['std_err']),
                'z': float(d['z']),
                'p': float(d['p']),
                'ci_low': float(d['ci_low']),
                'ci_high': float(d['ci_high']),
            })

df = pd.DataFrame(rows)

term = 'D-p_S(Z)_shifted'
s = df[df['name'] == term].set_index('run')['coef']
median_val = s.median()
run_with_median = (s - median_val).abs().idxmin()

blp_median_coef = (df[df['run'] == run_with_median]
       .reset_index(drop=True))

blp_median_coef 


---

In [ ]:
# ----- Flatten `results['gates']` (one row per split × group) -----
rows = []
for run in results['gates']:
    for d in run:
        rows.append({
            "group": int(d["group"]),
            "gamma": float(d["gamma"]),
            "std_err": float(d["std_err"]),
            "ci_low": float(d["ci_low"]),
            "ci_high": float(d["ci_high"]),
            # if your dict already has "p", we'll keep it; else we compute below
            "p": float(d["p"]) if "p" in d else np.nan,
        })

gdf = pd.DataFrame(rows)

# If p-values were not stored per split, approximate from gamma/std_err (two-sided normal)
if gdf["p"].isna().any():
    z = gdf["gamma"] / gdf["std_err"]
    gdf["p"] = 2 * (1 - norm.cdf(np.abs(z)))

# ----- Aggregate across splits EXACTLY like the R code (medians by group) -----
gates_summary = (
    gdf.groupby("group", as_index=False)
       .agg(gamma=("gamma", "median"),
       std_err=("std_err", "median"),
       ci_low=("ci_low", "median"),
       ci_high=("ci_high", "median"),
       p=("p", "median"),
       )

)

gates_summary


---

In [ ]:
rows = []
for run_i, run in enumerate(results['clan'], start=1):
    for d in run:
        r = {'run': run_i}
        r.update(d)
        rows.append(r)

df = pd.DataFrame(rows)

diff_col = 'Diff (G5 - G1)'

# drop rows with NaN diff (so medians/closest work cleanly)
df_ = df.dropna(subset=[diff_col]).copy()

# median diff per covariate
med = df_.groupby('covariate')[diff_col].median().rename('diff_median')

# find the row per covariate closest to its median
tmp = df_.merge(med, on='covariate', how='left')
idx = (tmp[diff_col] - tmp['diff_median']).abs().groupby(tmp['covariate']).idxmin()

clan = (tmp.loc[idx]
       .drop(columns=['diff_median'])
       .sort_values('covariate')
       .reset_index(drop=True))

clan

In [ ]:
#additional_feature_drop = ['vehicle_motorcycle_e','vehicle_toktok_e','vehicle_tricycle_e','skip_meal_e','has_heater_e','has_fridge_e','cart_e','bicycle_e']

In [ ]:
clan[clan['p (two-sided)'] < 0.05].shape[0]

In [ ]:
clan[clan['p (two-sided)'] < 0.05]